# Importe de librerias

In [ ]:
import pandas as pd
import json
import lightning as L
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
from dataset import LitPriceData
from trainer import LitTrainer

# Carga de monedas selecionadas

In [3]:
coins = pd.read_json('../data/selected_coins.json')
coins

,0
0,celestia
1,flare-networks
2,polygon-bridged-usdt-polygon
3,xdce-crowd-sale
4,injective-protocol
5,virtual-protocol
6,blockstack
7,sonic-3
8,binance-bridged-usdc-bnb-smart-chain
9,mantle-staked-ether


In [ ]:
def priceDataframe(name):
    jsonPath = f"../data/info_{name}.json"
    with open(jsonPath, "r") as f:
        data = json.load(f) 
        
    df = pd.DataFrame({
        "price_timestamp": [item[0] for item in data["prices"]],
        "price": [item[1] for item in data["prices"]],
        "market_caps_timestamp": [item[0] for item in data["market_caps"]],
        "market_caps": [item[1] for item in data["market_caps"]],
        "volume_timestamp": [item[0] for item in data["total_volumes"]],
        "volume": [item[1] for item in data["total_volumes"]]
    })
    
    df["price_timestamp"] = pd.to_datetime(df["price_timestamp"], unit="ms")
    #df["market_caps_timestamp"] = pd.to_datetime(df["market_caps_timestamp"], unit="ms")
    #df["volume_timestamp"] = pd.to_datetime(df["volume_timestamp"], unit="ms")

    df["hour"] = df["price_timestamp"].dt.hour
    #df["market_caps_timestamp_hour"] = df["market_caps_timestamp"].dt.hour
    #df["volume_timestamp_hour"] = df["volume_timestamp"].dt.hour

    df["dayweek"] = df["price_timestamp"].dt.dayofweek
    #df["market_caps_timestamp_dayweek"] = df["market_caps_timestamp"].dt.dayofweek
    #df["volume_timestamp_dayweek"] = df["volume_timestamp"].dt.dayofweek

    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)

    df["dayweek_cos"] = np.cos(2 * np.pi * df["hour"] / 7)
    df["dayweek_sin"] = np.sin(2 * np.pi * df["hour"] / 7)

    df.drop(columns=["price_timestamp", "market_caps_timestamp", "volume_timestamp"], inplace=True)
    df.drop(columns=["hour", "dayweek"], inplace=True)

    return df

In [ ]:
def splitData(name, sl=72, po=120):
    df = priceDataframe(name)

    X, y = [], []
    for i in range(len(df) - sl - po):
        X.append(df[i : i + sl])
        y.append(df[i + sl + po - 1])

    X = np.array(X)
    y = np.array(y)
    
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)   
    
    return X_tensor, y_tensor

In [ ]:
def trainModel(model, X, y, checkpoint_callback=None):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    dataModule = LitPriceData(X_train, y_train, X_val, y_val)
       
    if checkpoint_callback is not None:
        trainer = L.Trainer(max_epochs=50, accelerator="auto", callbacks=[checkpoint_callback])
    else:
        trainer = L.Trainer(max_epochs=50, accelerator="auto")

    trainer.fit(model=model, datamodule=dataModule)

In [ ]:
for i, row in coins.iterrows():    
    name = row[0]

    checkpoint_callback = ModelCheckpoint(
        dirpath = f"../models/{name}"
        filename = "checkpoints",
        save_top_k=2,
        monitor="val_loss",
        mode="min"        
    )

    #Change the values
    model = LitTrainer(hidden_size=256, num_layers=3, lr=1e-3)
    X, y = splitData(name)
    trainModel(model, X, y, checkpoint_callback)

celestia
flare-networks
polygon-bridged-usdt-polygon
xdce-crowd-sale
injective-protocol
virtual-protocol
blockstack
sonic-3
binance-bridged-usdc-bnb-smart-chain
mantle-staked-ether
stakewise-v3-oseth
optimism
pudgy-penguins
syrupusdc
story-2
